In [ ]:
import pandas as pd
import numpy as np
import ruptures as rpt
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets
import matplotlib.pyplot as plt

pio.templates.default = "seaborn"
pd.options.plotting.backend = "plotly"

%matplotlib inline

In [ ]:
%store -r df

In [ ]:
%run functions.ipynb

In [ ]:
def plot_weekdays(title, users):
    steps = users.groupby(['hour','series']).median().unstack('series')['steps']
    plot = steps.plot.line(markers=False, line_shape='spline', height=400, title=title)
    
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    for day in range(0,7):
        start = day * 24
        end = start + 23
        stepsBefore = steps['before'][start:end].sum()
        stepsAfter = steps['after'][start:end].sum()
        offset = start + 2
        
        annotate_diff(plot, offset, 600, 50, days[day], stepsBefore, stepsAfter)

    return go.FigureWidget(plot)

In [ ]:
users = df[df['work_age'] == False]
steps = users.groupby(['hour','series']).median().unstack('series')['steps']
plot = steps.plot.line(markers=False, line_shape='spline', height=400)

weekend_start = (24 * 5) - 1
ranges = [{'start':0, 'end': weekend_start, 'length': 5}, {'start': weekend_start, 'end': 167, 'length': 2}]

weekday_dec = 0
weekend_dec = 0

for range in ranges:
    start = range['start']
    end = range['end']
    stepsBefore = steps['before'][start:end].sum()
    stepsAfter = steps['after'][start:end].sum()
    if start == 0:
        weekday_dec = stepsAfter - stepsBefore
    else:
        weekend_dec = stepsAfter - stepsBefore
    
    annotate_diff(plot, start, 600, 50, '', stepsBefore, stepsAfter)
    # plot.add_vrect(x0=start, x1=end - 1, opacity=0.15, layer="below", line_width=2, line_dash="dash")

diff = weekend_dec + weekday_dec
plot.add_annotation(x=146, y=700, text='{:10.0f} (weekday)'.format(weekday_dec), showarrow=False, font=dict(
    color='black',
    size=14,
))
plot.add_annotation(x=146, y=650, text='{:10.0f} (weekend)'.format(weekend_dec), showarrow=False, font=dict(
    color='black',
    size=14,
))
plot.add_annotation(x=140, y=600, text='{:10.0f}'.format(diff), showarrow=False, font=dict(
    color='black',
    size=14,
))
    
plot.show()

In [ ]:

workers = df[df['work_age'] == True]
non_workers = df[df['work_age'] == False]

widgets.VBox([plot_weekdays('All', df), plot_weekdays('Workers', workers), plot_weekdays('Non workers', non_workers)])